In [86]:
from dataclasses import dataclass, field
from typing import Callable, List, Dict

import numpy as np
import pandas as pd
from dateutil.relativedelta import relativedelta
pd.options.plotting.backend = 'plotly'


@dataclass
class WalkforwardBacktester:
    """
    A Walkforward Backtester with Top-N strategy ensembling.
    Instead of picking a single winner, selects the top_n strategies
    and averages their returns for diversification.

    Attributes
    ----------
    pnl_df : pd.DataFrame
        DataFrame of PnL/returns for multiple strategies.
    lookback_period : str
        Lookback period for evaluating strategy performance (e.g., '24ME').
    rebalance_freq : str
        Frequency of rebalance (e.g., '3ME').
    top_n : int
        Number of top strategies to ensemble (default=3).
    score_func : Callable
        Function to score each strategy. Defaults to Sharpe ratio.
    """

    pnl_df: pd.DataFrame
    lookback_period: str
    rebalance_freq: str
    top_n: int = 3
    score_func: Callable[[pd.Series], float] = field(default=None)
    periods: List = None
    lookback_start_skip_check: bool = False
    rebalances: List[Dict[str, any]] = field(default_factory=list)

    def __post_init__(self):
        self.score_func = self.score_func or self._sharpe_ratio
        self.portfolio_pnl = pd.Series(index=self.pnl_df.index, dtype=np.float64)

    def _sharpe_ratio(self, returns: pd.Series) -> float:
        return returns.mean() / returns.std() if returns.std() != 0 else 0

    def _select_top_n_strategies(self, lookback_df: pd.DataFrame):
        """Select the top_n strategies by score and return their names + scores."""
        strategy_scores = lookback_df.apply(self.score_func)
        top_strategies = strategy_scores.nlargest(self.top_n).index.tolist()
        return top_strategies, strategy_scores

    def run_backtest(self) -> pd.Series:
        """
        Run the walkforward backtest. At each rebalance, select the top_n strategies
        and average their returns for the forward period.
        """
        freq_letter = self.rebalance_freq[-1] if self.rebalance_freq[-1] != "E" else self.rebalance_freq[-2]
        assert freq_letter == "M", "rebalance freq must be in months"

        self.periods = pd.date_range(
            start=self.pnl_df.index[0],
            end=self.pnl_df.index[-1] + relativedelta(months=int(self.rebalance_freq.replace('ME','').replace('M',''))),
            freq=self.rebalance_freq
        )

        for start_date, end_date in zip(self.periods[:-1], self.periods[1:]):
            lookback_end = start_date - pd.Timedelta(days=1)
            lookback_start = lookback_end - relativedelta(months=int(self.lookback_period.replace('ME','').replace('M','')))

            if (lookback_start < self.pnl_df.index[0]) and (not self.lookback_start_skip_check):
                continue

            lookback_df = self.pnl_df.loc[lookback_start:lookback_end]
            top_strats, strategy_scores = self._select_top_n_strategies(lookback_df)

            # Average the returns of the top_n strategies (equal weight ensemble)
            ensemble_returns = self.pnl_df[top_strats].loc[start_date:end_date].mean(axis=1)

            self.rebalances.append({
                'start_date': start_date,
                'end_date': end_date,
                'lookback_start': lookback_start,
                'lookback_end': lookback_end,
                'top_strategies': top_strats,
                'strategy_scores': strategy_scores,
                'lookback_df': lookback_df,
                'ensemble_mean_returns': ensemble_returns.mean(),
                'ensemble_std': ensemble_returns.std()
            })

            self.portfolio_pnl.loc[start_date:end_date] = ensemble_returns

        return self.portfolio_pnl

    def get_rebalance_details(self) -> pd.DataFrame:
        return pd.DataFrame(self.rebalances)




In [87]:
"""
NOTE:
This is a very basic draft strategy for illustrative purposes. This Strategy goes long or short based on the current strat selection - momentum or 
mean reversion and last n days return. 
- The walkforward backtester evaluates multiple strategies over a rolling lookback period.
- At each rebalance, the best performing strategy (based on Sharpe ratio by default) is applied forward.
- The strategy here (simple momentum vs mean reversion with fixed lookbacks) performs poorly as can be seen in performance charts for walfforward
- Goal: use this framework to design a more robust strategy with better walkforward performance.
"""


# -------------------------------
# Example Usage: Brent Index
# -------------------------------
df = pd.read_excel(r"brent_index.xlsx", parse_dates=['date'], index_col='date')
rets = df['CO1 Comdty'].pct_change()
tcost = 0.00015  # Transaction cost per trade

# -------------------------------
# Volatility Scaling (Risk Parity)
# -------------------------------
target_annual_vol = 0.15  # Target 15% annualized volatility
rolling_vol = rets.rolling(20).std() * np.sqrt(252)  # 20-day annualized vol
vol_scalar = (target_annual_vol / rolling_vol).clip(0.25, 3.0)  # Cap between 0.25x and 3x

# -------------------------------
# Regime Detection (200-day SMA)
# -------------------------------
sma_200 = df['CO1 Comdty'].rolling(200).mean()
trending = df['CO1 Comdty'] > sma_200       # Price above 200 SMA = trending up
sideways = ~trending                         # Price below 200 SMA = sideways/down

# -------------------------------
# Helper: Minimum Holding Period Filter
# -------------------------------
def apply_holding_period(signal, min_hold=5):
    """Enforce a minimum holding period to reduce excessive trading.
    Once a position is taken, it must be held for at least `min_hold` days
    before the signal can change."""
    filtered = signal.copy()
    last_trade_idx = -min_hold  # allow first trade immediately
    current_pos = 0
    for i in range(len(filtered)):
        if pd.isna(filtered.iloc[i]):
            filtered.iloc[i] = current_pos
            continue
        if filtered.iloc[i] != current_pos:
            if (i - last_trade_idx) >= min_hold:
                current_pos = filtered.iloc[i]
                last_trade_idx = i
            else:
                filtered.iloc[i] = current_pos  # keep old position
        else:
            filtered.iloc[i] = current_pos
    return filtered

min_hold = 5  # Minimum 5-day holding period (tune this parameter)

def apply_stop_loss_take_profit(signal, prices, stop_loss=-0.02, take_profit=0.04):
    """Apply stop-loss and take-profit rules to a signal.
    When a position is entered, track cumulative return from entry price.
    Exit (go flat) if return hits stop_loss or take_profit.
    Stay flat until signal changes direction for a new entry.
    
    Parameters:
        signal: pd.Series of positions (+1, -1, or 0)
        prices: pd.Series of asset prices (same index as signal)
        stop_loss: exit threshold for losses (e.g., -0.02 = -2%)
        take_profit: exit threshold for gains (e.g., 0.04 = +4%)
    """
    filtered = signal.copy().astype(float)
    entry_price = None
    current_pos = 0
    stopped_out = False  # True when SL/TP triggered, waiting for new signal
    
    for i in range(len(filtered)):
        raw_signal = filtered.iloc[i]
        price = prices.iloc[i]
        
        if pd.isna(price) or pd.isna(raw_signal):
            filtered.iloc[i] = current_pos
            continue
        
        # If we're stopped out, stay flat until signal changes
        if stopped_out:
            if raw_signal != current_pos and raw_signal != 0:
                # New signal direction -> re-enter
                current_pos = raw_signal
                entry_price = price
                stopped_out = False
            else:
                filtered.iloc[i] = 0
                continue
        
        # Check if entering a new position
        if current_pos == 0 and raw_signal != 0:
            current_pos = raw_signal
            entry_price = price
            filtered.iloc[i] = current_pos
            continue
        
        # If in a position, check SL/TP
        if current_pos != 0 and entry_price is not None and entry_price != 0:
            pnl_pct = current_pos * (price - entry_price) / entry_price
            
            if pnl_pct <= stop_loss:  # Stop loss hit
                filtered.iloc[i] = 0
                current_pos = 0
                entry_price = None
                stopped_out = True
                continue
            elif pnl_pct >= take_profit:  # Take profit hit
                filtered.iloc[i] = 0
                current_pos = 0
                entry_price = None
                stopped_out = True
                continue
        
        # Check for signal direction change (new entry)
        if raw_signal != current_pos and raw_signal != 0:
            current_pos = raw_signal
            entry_price = price
        
        filtered.iloc[i] = current_pos
    
    return filtered

stop_loss = -0.02   # -2% stop loss
take_profit = 0.04  # +4% take profit


# ------------------------------- # Define multiple strategies # ----------
lookback_days = [2, 4, 8, 16, 32, 64, 128, 256] 
pnl_strategies = pd.DataFrame(index=rets.index) 
for m in lookback_days: #rolling returns till yesterday, Shift(1) below to remove look ahead bias 
    rolling_ret = (1 + rets).rolling(window=m).apply(np.prod, raw=True).shift(1) - 1 
    
    # Momentum: go long if past return > 0, else short 
    momentum_signal = pd.Series(np.where(rolling_ret > 0, 1, -1), index=rets.index) 
    momentum_signal = apply_holding_period(momentum_signal, min_hold)
    momentum_signal = apply_stop_loss_take_profit(momentum_signal, df['CO1 Comdty'], stop_loss, take_profit)
    # Apply regime filter: momentum only allowed in trending markets
    momentum_signal = momentum_signal.where(trending.shift(1), 0)
    # Apply volatility scaling
    pnl_strategies[f"{m}D_momentum"] = (momentum_signal * vol_scalar) * rets - tcost * abs(momentum_signal.diff()) 
    
    # Mean Reversion: go short if past return > 0, else long 
    meanrev_signal = pd.Series(np.where(rolling_ret > 0, -1, 1), index=rets.index) 
    meanrev_signal = apply_holding_period(meanrev_signal, min_hold)
    meanrev_signal = apply_stop_loss_take_profit(meanrev_signal, df['CO1 Comdty'], stop_loss, take_profit)
    # Apply regime filter: mean reversion only in sideways markets
    meanrev_signal = meanrev_signal.where(sideways.shift(1), 0)
    # Apply volatility scaling
    pnl_strategies[f"{m}D_meanrev"] = (meanrev_signal * vol_scalar) * rets - tcost * abs(meanrev_signal.diff())
    
# -------------------------------
# Moving Average Crossover Strategies
# -------------------------------
ma_pairs = [(2,10),(5, 20), (10, 50), (20, 100), (50, 200), (100,300)]  # (short_window, long_window)
for short_w, long_w in ma_pairs:
    short_ma = df['CO1 Comdty'].rolling(window=short_w).mean()
    long_ma = df['CO1 Comdty'].rolling(window=long_w).mean()
    # Go long when short MA > long MA, else short
    ma_signal = pd.Series(np.where(short_ma.shift(1) > long_ma.shift(1), 1, -1), index=rets.index)
    ma_signal = apply_holding_period(ma_signal, min_hold)
    ma_signal = apply_stop_loss_take_profit(ma_signal, df['CO1 Comdty'], stop_loss, take_profit)
    pnl_strategies[f"{short_w}_{long_w}_ma_crossover"] = (ma_signal * vol_scalar) * rets - tcost * abs(ma_signal.diff())

# # -------------------------------
# # Bollinger Band Reversion Strategies
# # -------------------------------
# bb_windows = [20, 50]
# for w in bb_windows:
#     mid = df['CO1 Comdty'].rolling(window=w).mean()
#     std = df['CO1 Comdty'].rolling(window=w).std()
#     upper = mid + 2 * std
#     lower = mid - 2 * std
#     price = df['CO1 Comdty']
#     # Long when price below lower band, short when above upper, else flat->use previous
#     bb_signal = pd.Series(np.where(price.shift(1) < lower.shift(1), 1,
#                          np.where(price.shift(1) > upper.shift(1), -1, np.nan)), index=rets.index)
#     bb_signal = bb_signal.ffill().fillna(0)
#     bb_signal = apply_holding_period(bb_signal, min_hold)
#     bb_signal = apply_stop_loss_take_profit(bb_signal, df['CO1 Comdty'], stop_loss, take_profit)
#     pnl_strategies[f'{w}D_bollinger'] = (bb_signal * vol_scalar) * rets - tcost * abs(bb_signal.diff())

# -------------------------------
# RSI Strategies
# -------------------------------
rsi_windows = [14, 28]
for w in rsi_windows:
    delta = df['CO1 Comdty'].diff()
    gain = delta.where(delta > 0, 0).rolling(window=w).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=w).mean()
    rs = gain / loss
    rsi = 100 - (100 / (1 + rs))
    # Long when RSI < 30 (oversold), short when RSI > 70 (overbought)
    rsi_signal = pd.Series(np.where(rsi.shift(1) < 30, 1,
                          np.where(rsi.shift(1) > 70, -1, np.nan)), index=rets.index)
    rsi_signal = rsi_signal.ffill().fillna(0)
    rsi_signal = apply_holding_period(rsi_signal, min_hold)
    rsi_signal = apply_stop_loss_take_profit(rsi_signal, df['CO1 Comdty'], stop_loss, take_profit)
    pnl_strategies[f'{w}D_rsi'] = (rsi_signal * vol_scalar) * rets - tcost * abs(rsi_signal.diff())

# -------------------------------
# Donchian Breakout Strategies
# -------------------------------
# breakout_windows = [20, 55]
# for w in breakout_windows:
#     high_w = df['CO1 Comdty'].rolling(window=w).max()
#     low_w = df['CO1 Comdty'].rolling(window=w).min()
#     price = df['CO1 Comdty']
#     # Long on new high, short on new low
#     brk_signal = pd.Series(np.where(price.shift(1) >= high_w.shift(2), 1,
#                           np.where(price.shift(1) <= low_w.shift(2), -1, np.nan)), index=rets.index)
#     brk_signal = brk_signal.ffill().fillna(0)
#     brk_signal = apply_holding_period(brk_signal, min_hold)
#     brk_signal = apply_stop_loss_take_profit(brk_signal, df['CO1 Comdty'], stop_loss, take_profit)
#     pnl_strategies[f'{w}D_breakout'] = (brk_signal * vol_scalar) * rets - tcost * abs(brk_signal.diff())

# -------------------------------
# Run Walkforward Backtest
# -------------------------------
backtester = WalkforwardBacktester(
    pnl_df=pnl_strategies,
    lookback_period='24M',
    rebalance_freq='3M',
    top_n=3  # Ensemble top 3 strategies
)

portfolio_pnl = backtester.run_backtest()

# -------------------------------
# Inspect Rebalance Decisions
# -------------------------------
rebalance_details = backtester.get_rebalance_details()

# -------------------------------
# Portfolio Performance
# -------------------------------
portfolio_cum = (1 + portfolio_pnl).cumprod()
portfolio_cum.plot(title="Walkforward Portfolio Cumulative Returns (Momentum vs Mean Reversion)")

In [88]:
pnl_strategies.cumsum().plot()

In [89]:
# -------------------------------
# Overall Strategy Performance Summary
# -------------------------------
import warnings
warnings.filterwarnings('ignore')

pnl = portfolio_pnl.dropna()
trading_days = 252

# Core metrics
total_return = (1 + pnl).prod() - 1
ann_return = (1 + total_return) ** (trading_days / len(pnl)) - 1
ann_vol = pnl.std() * np.sqrt(trading_days)
sharpe = ann_return / ann_vol if ann_vol != 0 else 0

# Max Drawdown
cum = (1 + pnl).cumprod()
peak = cum.cummax()
drawdown = (cum - peak) / peak
max_dd = drawdown.min()

# Calmar Ratio (ann return / max drawdown)
calmar = ann_return / abs(max_dd) if max_dd != 0 else 0

# Win rate
win_rate = (pnl > 0).sum() / len(pnl)

# Profit Factor
gross_profit = pnl[pnl > 0].sum()
gross_loss = abs(pnl[pnl < 0].sum())
profit_factor = gross_profit / gross_loss if gross_loss != 0 else float('inf')

print('=' * 50)
print('  WALKFORWARD STRATEGY - PERFORMANCE SUMMARY')
print('=' * 50)
print(f'  Total Return:        {total_return:>10.2%}')
print(f'  Annualized Return:   {ann_return:>10.2%}')
print(f'  Annualized Volatility:{ann_vol:>9.2%}')
print(f'  Sharpe Ratio:        {sharpe:>10.2f}')
print(f'  Max Drawdown:        {max_dd:>10.2%}')
print(f'  Calmar Ratio:        {calmar:>10.2f}')
print(f'  Win Rate:            {win_rate:>10.2%}')
print(f'  Profit Factor:       {profit_factor:>10.2f}')
print('=' * 50)
print(f'      >>> OVERALL SHARPE RATIO: {sharpe:.2f} <<<\n')
print(f'(The single best measure of risk-adjusted performance)')


  WALKFORWARD STRATEGY - PERFORMANCE SUMMARY
  Total Return:           107.63%
  Annualized Return:       11.90%
  Annualized Volatility:    7.42%
  Sharpe Ratio:              1.60
  Max Drawdown:            -7.53%
  Calmar Ratio:              1.58
  Win Rate:                36.96%
  Profit Factor:             1.40
      >>> OVERALL SHARPE RATIO: 1.60 <<<

(The single best measure of risk-adjusted performance)


In [90]:
pnl_strategies.head()

,2D_momentum,2D_meanrev,4D_momentum,4D_meanrev,8D_momentum,8D_meanrev,16D_momentum,16D_meanrev,32D_momentum,32D_meanrev,...,256D_momentum,256D_meanrev,2_10_ma_crossover,5_20_ma_crossover,10_50_ma_crossover,20_100_ma_crossover,50_200_ma_crossover,100_300_ma_crossover,14D_rsi,28D_rsi
date,,,,,,,,,,,,,,,,,,,,,
2016-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016-01-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016-01-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016-01-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016-01-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
